In [1]:
import os
#import psutil
import rasterio
from rasterio.features import rasterize
import numpy as np
import geopandas as gpd
import pandas as pd
from sklearn.feature_extraction import image
from shapely.geometry import Polygon
from shapely.ops import cascaded_union
from tqdm import tqdm
import cv2

In [5]:
path_GeoJSON = './suite_data_pie/GeoJSON/'
path_GeoTIFF = './suite_data_pie/GeoTIFF/'

list_imgs = os.listdir(path_GeoTIFF)
list_masks = os.listdir(path_GeoJSON)
data_dict = {}

for img in list_imgs: 
    name = img.split('_ortho')[0]
    data_dict[img] = [mask for mask in list_masks if name in mask ][0]

In [6]:
# Liste des features d'intérêt et color mapping (1 canal nuances de gris de 1 à 255, 0 est background)
color_mapping = {
    'Apron.apronelement': 1,
    'Runway.runwayelement': 2,
    'ServiceRoad.serviceroad': 3,
}
features_list = ['Apron.apronelement','Runway.runwayelement','ServiceRoad.serviceroad']

In [7]:
def polygon_df(filepath):
    df = gpd.read_file(filepath)
    columns_todrop=['originated', 'readonly',
           'notvalidated', 'lock', 'elev', 'hacc', 'iata', 'name', 'idarpt',
           'acft', 'idnumber', 'termref', 'pcn', 'restacft', 'status', 'surftype',
           'length', 'width', 'color', 'direc', 'style', 'rwymktyp', 'asda',
           'availPavedSurfFromThr', 'brngmag', 'brngtrue', 'cat', 'ellipse',
           'geound', 'lda', 'rops_landing_length', 'rwyslope', 'tdze', 'tdzslope',
           'thrtype', 'toda', 'tora', 'vasis', 'bridge', 'gsurftyp', 'runwayexit',
           'imagery_date']
    df = df.drop(columns = [c for c in df.columns if c in columns_todrop])
    
    return df[df["geometry"].geom_type == 'Polygon']

# Fonctions de création de masques par lecture du GeoJson

def poly_from_utm(polygon, transform):
    poly_pts = []
    
    # make a polygon from multipolygon
    poly = cascaded_union(polygon)
    for i in np.array(poly.exterior.coords):
        
        # transfrom polygon to image crs, using raster meta
        poly_pts.append(~transform * tuple(i))
        
    # make a shapely Polygon object
    new_poly = Polygon(poly_pts)
    return new_poly

# creating binary mask for field/not_filed segmentation.
def create_mask(features_list,src,df):
    nb_class=len(features_list)
    im_size = (src.meta['height'], src.meta['width'])
    whole_mask = np.zeros([src.meta['height'],src.meta['width']])

    for i,f in enumerate(features_list):
        poly_shp = []
        data_geom = df[df['feattype'] == {'value': f}].geometry
        for g in data_geom.values:

            poly = poly_from_utm(g, src.meta['transform'])
            poly_shp.append(poly)

            if g.geom_type == 'Polygon':
                poly = poly_from_utm(g, src.meta['transform'])
                poly_shp.append(poly)
            else:
                for p in g:
                    poly = poly_from_utm(p, src.meta['transform'])
                    poly_shp.append(poly)
        if poly_shp:
            mask = rasterize(shapes=poly_shp,out_shape=im_size)*color_mapping[f]
            #print(mask.shape)
            whole_mask = whole_mask + mask
           
    return whole_mask

def create_mask_multi_channel(features_list,src,df):
    nb_class=len(features_list)
    im_size = (src.meta['height'], src.meta['width'])
    whole_mask = np.zeros([src.meta['height'],src.meta['width'],nb_class])

    for i,f in enumerate(features_list):
        poly_shp = []
        data_geom = df[df['feattype'] == {'value': f}].geometry
        for g in data_geom.values:

            poly = poly_from_utm(g, src.meta['transform'])
            poly_shp.append(poly)

            if g.geom_type == 'Polygon':
                poly = poly_from_utm(g, src.meta['transform'])
                poly_shp.append(poly)
            else:
                for p in g:
                    poly = poly_from_utm(p, src.meta['transform'])
                    poly_shp.append(poly)
        if poly_shp:
            mask = rasterize(shapes=poly_shp,out_shape=im_size)*color_mapping[f]
            #print(mask.shape)
            whole_mask[:,:,i] = mask
           
    return whole_mask


In [8]:
grid_unit = (256,256)
os.makedirs('./data/img', exist_ok=True)
os.makedirs('./data/mask', exist_ok=True)


for filename in data_dict: 
    filepath = os.path.join(path_GeoJSON,data_dict[filename])
    src = rasterio.open(path_GeoTIFF+"{}".format(filename))
    img = src.read().transpose((1,2,0))
    src.close()
    try:
        df = polygon_df(filepath)
        whole_mask = create_mask_multi_channel(features_list,src,df)
        mask = whole_mask.max(axis=len(features_list)-1)
        for i in range(img.shape[0]//grid_unit[0] + 1):
            for j in range(img.shape[1]//grid_unit[1] + 1):
                img_window = img[i*grid_unit[0]:(i+1)*grid_unit[0],j*grid_unit[1]:(j+1)*grid_unit[1],:]
                mask_window = mask[i*grid_unit[0]:(i+1)*grid_unit[0],j*grid_unit[1]:(j+1)*grid_unit[1]]
                name_window = filename.split('.')[0] + '_{}_{}'.format(i,j)
                if not np.sum(np.any(img_window==0)) and img_window.shape[0] == grid_unit[0] and img_window.shape[1] == grid_unit[1]:
                    cv2.imwrite('./data/img/'+ name_window + '.png',img_window)
                    cv2.imwrite('./data/mask/'+ 'mask_' + name_window + '.png',mask_window)
        print('{} saved'.format(filename))
    except: 
        print('{} error occured'.format(filename))
    #print('Free memory: {}'.format(dict(psutil.virtual_memory()._asdict())['free']))

    

FMEE_ortho.tif saved
Free memory: 3807621120
